 # STEP 4
 ### make your "matmul_OS_proj3" and "matmul_WS_proj3"
 ### You can check your score with STEP4_test() function.

In [1]:
from pynq import PL
PL.reset()

In [2]:
from pynq import Overlay
ol = Overlay("bitstream_team4.bit")

In [3]:
ol?

In [4]:
import pynq
# You can change the clock frequency #
pynq.ps.Clocks.fclk0_mhz = 25 #(unit: MHz)
print(pynq.ps.Clocks.fclk0_mhz)

25.0


#### This is the instance of bram controller, you can insert the values via using this.

In [5]:
bram_sp_arr = ol.axi_bram_ctrl_0.mmio.array
bram_a_arr = ol.axi_bram_ctrl_1.mmio.array
bram_w_arr = ol.axi_bram_ctrl_2.mmio.array
bram_o_arr = ol.axi_bram_ctrl_3.mmio.array

# TODO
### Put the matrix in BRAM the way you designed it.
### Set the parameters in SP_BRAM (you can add more parameters for project 3 if you want)


In [6]:
import numpy as np
import random 
import time

def matmul_OS_proj3  (mat1: np.array, mat2: np.array):
    assert(mat1.shape[1] == mat2.shape[0])
    TILING = 8

    mat1_uint = mat1.astype(np.uint32)
    mat2_uint = mat2.astype(np.uint32)

    # ======matrix into BRAM ===== #
    M = ((mat1.shape[0] + TILING - 1) // TILING) * TILING
    K = ((mat1.shape[1] + TILING - 1) // TILING) * TILING
    N = ((mat2.shape[1] + TILING - 1) // TILING) * TILING
    mat1_pad = np.zeros((M, K)).astype(np.uint32)
    mat2_pad = np.zeros((K, N)).astype(np.uint32)
    mat1_pad[0:mat1.shape[0], 0:mat1.shape[1]] = mat1_uint
    mat2_pad[0:mat2.shape[0], 0:mat2.shape[1]] = mat2_uint
    bram_a_arr[0:M * K] = mat1_pad.flatten()
    bram_w_arr[0:K * N] = mat2_pad.flatten()

    
    #=============== HW RUNNIG ==================#
    # set Special Memory to specify M,K,N and mode (1: OS)    
    bram_sp_arr[1] = 1
    bram_sp_arr[2] = M
    bram_sp_arr[3] = K
    bram_sp_arr[4] = N
    # start (set sp(addr0) = 1)
    bram_sp_arr[0] = 1
    #while until sp(addr0) => 0 or sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        pass
    # get data from BARM_O
    bram_sp_arr[25] = 0
    
    #=============== HW END =====================#


    return bram_o_arr[0:M * N].astype(np.int32).reshape(M, N)[0:mat1.shape[0], 0:mat2.shape[1]]

def matmul_WS_proj3 (mat1: np.array, mat2: np.array):
    assert(mat1.shape[1] == mat2.shape[0])
    TILING = 8

    mat1_uint = mat1.astype(np.uint32)
    mat2_uint = mat2.astype(np.uint32)

    # ======matrix into BRAM ===== #
    M = ((mat1.shape[0] + TILING - 1) // TILING) * TILING
    K = ((mat1.shape[1] + TILING - 1) // TILING) * TILING
    N = ((mat2.shape[1] + TILING - 1) // TILING) * TILING
    mat1_pad = np.zeros((M, K)).astype(np.uint32)
    mat2_pad = np.zeros((K, N)).astype(np.uint32)
    mat1_pad[0:mat1.shape[0], 0:mat1.shape[1]] = mat1_uint
    mat2_pad[0:mat2.shape[0], 0:mat2.shape[1]] = mat2_uint
    bram_a_arr[0:M * K] = mat1_pad.flatten()
    bram_w_arr[0:K * N] = mat2_pad.flatten()

    
    #=============== HW RUNNIG ==================#
    # set Special Memory to specify M,K,N and mode (1: OS)    
    bram_sp_arr[1] = 0
    bram_sp_arr[2] = M
    bram_sp_arr[3] = K
    bram_sp_arr[4] = N
    # start (set sp(addr0) = 1)
    bram_sp_arr[0] = 1
    #while until sp(addr0) => 0 or sp(addr100) =>1 
    while(bram_sp_arr[25] != 1):
        pass
    # get data from BARM_O
    bram_sp_arr[25] = 0
    
    #=============== HW END =====================#
    
    
    return bram_o_arr[0:M * N].astype(np.int32).reshape(M, N)[0:mat1.shape[0], 0:mat2.shape[1]]

In [12]:
import random

def test_OS():
    score = 0
    num_test = 0
    ## OUTPUT STATIONARY TEST
    for _ in range (1000):
        M = random.randint(1,40)
        K = random.randint(1,40)
        N = random.randint(1,40)
        num_test = num_test +1
        mat1 = np.random.randint(-(1<<12), 1<<12 ,size=(M,K)) 
        mat2 = np.random.randint(-(1<<12), 1<<12 ,size=(K,N)) 
        CPU = mat1@mat2
        FPGA = matmul_OS_proj3(mat1, mat2)

        if(np.equal(CPU,FPGA).all()):
            score = score +1
        else:
            print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
            print(CPU)
            print(FPGA)
            print(np.equal(FPGA, CPU))
            
    return score/num_test * 100


def test_WS():
    score = 0
    num_test = 0
    ## WEIGHT STATIONARY TEST
    for iter_ in range (1000):
        
        M = random.randint(1,40)
        K = random.randint(1,40)
        N = random.randint(1,40)
        num_test = num_test +1
        mat1 = np.random.randint(-(1<<12), 1<<12 ,size=(K,M))
        mat2 = np.random.randint(-(1<<12), 1<<12 ,size=(M,N))
        FPGA = matmul_WS_proj3(mat1, mat2)
        CPU = mat1@mat2
        if(np.equal(FPGA, CPU).all()):
            score = score + 1
        else:
            print("M: "+str(M) +" N:  "+str(K) +" K: "+str(N) +" ")
            print(CPU)
            print(FPGA)
            print(np.equal(FPGA, CPU))
    return score/num_test*100

In [13]:
test_OS()

100.0

In [14]:
test_WS()

100.0

In [10]:
def STEP4_test():
    print(" SCORE : " + str((test_WS() + test_OS()) * 0.5) + " / 100 " )

In [11]:
STEP4_test()

 SCORE : 100.0 / 100 
